# Cookies & Code - Testing!

In [1]:
# This needs to be at the beginning of any notebook using ipytest.
import ipytest
ipytest.autoconfig()

## Example Set 1 

A test is just a function with some assert statement. If this assert statement results in a true statement - the test passes! If it results in a failure, or an error occurs, the test will fail.

Here are a couple of examples:

In [53]:
%%ipytest -qq
# The above magic is what will cause the tests to actually run!
# What happens if you remove it?

def test_example():
    assert [1, 2, 3] == [1, 2, 3]

.                                                                                            [100%]


In [54]:
%%ipytest -qq

def test_will_fail():
    assert False

F                                                                                            [100%]
============================================= FAILURES =============================================
__________________________________________ test_will_fail __________________________________________

    def test_will_fail():
>       assert False
E       assert False

/var/folders/vv/d9ncb4ms2x1gl0mmkrvk8m7c0000gp/T/ipykernel_64744/2664154573.py:2: AssertionError
===================================== short test summary info ======================================
FAILED t_6651fbd894764815bb8c2b5f3a5d5116.py::test_will_fail - assert False


In [55]:
%%ipytest -qq

def this_test_will_not_run():
    assert False

Let's look at a slightly more interesting use case. We have a function "example_func" and we want to write some unit tests for it. 

Try running the tests. They should fail! Can you fix them?

In [4]:
def example_func(x: int, y: int) -> int:
    return x + y

In [52]:
%%ipytest -qq

def test_example():
    assert [1, 2, 3] == [1, 2, 3]

def test_example_func():
    ## Arrange
    x = 10
    y = 15

    ## Act
    output = example_func(x,  y)

    ## Assert
    assert output == 25

def test_example_func_failing_test():
    ## This test fails! Can you fix it?
    ## NB: Requires changing 1 line.

    ## Arrange
    x = 20
    y = 15

    ## Act
    output = example_func(x, y)

    ## Assert
    assert output == 20

def test_example_func_fails_with_none():
    ## This test fails! What is the problem?
    ## NB: Requires changing 1 line.
    
    ## Arrange
    x = None
    y = 2

    ## Act & Assert
    with pytest.raises(ValueError):
        output = example_func(x, y)

    
    

..FF                                                                                         [100%]
============================================= FAILURES =============================================
__________________________________ test_example_func_failing_test __________________________________

    def test_example_func_failing_test():
        ## This test fails! Can you fix it?
    
        ## Arrange
        x = 20
        y = 15
    
        ## Act
        output = example_func(x, y)
    
        ## Assert
>       assert output == 20
E       assert 35 == 20

/var/folders/vv/d9ncb4ms2x1gl0mmkrvk8m7c0000gp/T/ipykernel_64744/4040631578.py:29: AssertionError
________________________________ test_example_func_fails_with_none _________________________________

    def test_example_func_fails_with_none():
        ## This test fails! What is the problem?
    
        ## Arrange
        x = None
        y = 2
    
        ## Act & Assert
        with pytest.raises(ValueError):
>      

## Example 2

This tests a function that is made up of one public function and two private functions.

If we change from implementation 1 to implementation 2 - what happens to the tests?

What can we learn from this?

In [13]:
def _second_step(x: int, y: int) -> int:
    return x * y

def _first_step(x: int, y: int) -> int:
    return x + y

## Implementation One
def two_step_example(x: int, y: int) -> int:
    z = _first_step(x, y)
    z2 = _second_step(z, y)
    return z2

## Implementation Two
## Uncomment this and comment Implementation One when you're ready.
# def two_step_example(x: int, y: int) -> int:
#     z = x + y
#     z2 = z * y
#     return z2


In [20]:
%%ipytest -qq

def test_example():
    ## Arrange
    x = 2
    y = 5
    
    ## Run
    output = two_step_example(x, y)

    ## Assert
    assert output == 35

def test_example_first_step():
    ## Arrange
    x = 2
    y = 5

    ## Run
    output = _first_step(x, y)

    ## Assert
    assert output == 7

def test_example_second_step():
    ## Arrange
    x = 2
    y = 5

    ## Run 
    output = _second_step(x, y)

    ## Assert
    assert output == 10



...                                                                                          [100%]


What's wrong with the above?

The tests are coupled to the implementation!

# Fancy things - to look at in your own time

## pytest.mark.parameterize

Allows the same test function to run for many input / output pairs.


In [21]:
def square_number(x: int) -> int:
    return x ** 2

In [25]:
%%ipytest -qq
import pytest

@pytest.mark.parametrize("input_num,expected_output", [
    (2, 4), (-2, 4), (8, 64)
])
def test_square_number(input_num, expected_output):
    ## Arrange
    ## Nothing to do here

    ## Act
    output = square_number(input_num)

    ## Assert
    assert output == expected_output

...                                                                                          [100%]


## Fixtures

Allow re-use of setup objects that you use again and again - for instance reading an input file. Here's an example from pycodif.

In [ ]:
## Just for an example - this won't run.
## The "example_frame" code is executed before each test
## and the output passed in the "example_frame" argument.
class TestCODIFFrame:

    @pytest.fixture()
    def example_frame(self):
        with open("tests/test_files/test_codif.codif", "rb") as f:
            codif = CODIFFrame(f)
        return codif

    def test_data_parsing(self, example_frame):
        assert hasattr(example_frame, "header")
        assert hasattr(example_frame, "data_array")
        assert hasattr(example_frame, "sample_timestamps")

    def test_data_values(self, example_frame):
        assert isinstance(example_frame.data_array, np.ndarray)
        assert example_frame.data_array.dtype == np.dtype("complex64")
        assert example_frame.data_array[0, 0] == -23 + 45j
        assert example_frame.data_array[0, -1] == 113 - 89j
        assert example_frame.data_array[-1, 0] == 45j
        assert example_frame.data_array[-1, -1] == -43 + 58j

## Example 3 - Refactoring

I have some code below that reads in some data from a file, and constructs some Star objects with some characteristics.

In [31]:
import csv
from typing import List

class Star:
    def __init__(self, name: str, distance: float, luminosity: float):
        self.name = name
        self.distance = distance
        self.luminosity = luminosity

def process_astronomy_data(filename: str) -> List[Star]:
    """
    Reads a CSV file of star data and returns a list of Star objects.

    Args:
        filename (str): Path to the CSV file containing star data.
    
    Returns:
        List[Star]: A list of Star objects created from the CSV file.
    """
    stars = []
    with open(filename, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            name = row['Name']
            distance = float(row['Distance'])
            luminosity = float(row['Luminosity'])
            star = Star(name, distance, luminosity)
            stars.append(star)
    
    return stars



What makes testing the above function hard?

In [41]:
### Refactored version

from typing import List, Dict

def read_star_data(filename: str) -> List[Dict[str, str]]:
    """
    Reads a CSV file and returns the data as a list of dictionaries.
    
    Args:
        filename (str): Path to the CSV file containing star data.
    
    Returns:
        List[Dict[str, str]]: The raw data from the CSV as a list of dictionaries.
    """
    with open(filename, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        return [row for row in reader]

def create_star_objects(data: List[Dict[str, str]]) -> List[Star]:
    """
    Converts raw star data into Star objects.
    
    Args:
        data (List[Dict[str, str]]): Raw data containing star information.
    
    Returns:
        List[Star]: A list of Star objects created from the raw data.
    """
    stars = []
    for row in data:
        name = row['Name']
        distance = float(row['Distance'])
        luminosity = float(row['Luminosity'])
        star = Star(name, distance, luminosity)
        stars.append(star)
    return stars

def process_astronomy_data(filename: str) -> List[Star]:
    """
    Processes the astronomy data by reading and creating Star objects.
    
    Args:
        filename (str): Path to the CSV file containing star data.
    
    Returns:
        List[Star]: A list of Star objects.
    """
    raw_data = read_star_data(filename)
    return create_star_objects(raw_data)

In [56]:
%%ipytest -qq
from unittest.mock import patch, mock_open

sample_data = [
    {"Name": "Star A", "Distance": "10", "Luminosity": "1000"},
    {"Name": "Star B", "Distance": "20", "Luminosity": "2000"}
]

@pytest.mark.parametrize("data,expected_stars", [
    (sample_data, [
        Star("Star A", 10.0, 1000.0),
        Star("Star B", 20.0, 2000.0)
    ])
])
def test_create_star_objects(data, expected_stars):
    stars = create_star_objects(data)
    for star, expected_star in zip(stars, expected_stars):
        assert star.name == expected_star.name
        assert star.distance == expected_star.distance
        assert star.luminosity == expected_star.luminosity

# Using a dummy file to test data ingestion.
def test_process_astronomy_data():
    # This test fails. Can you use the debugger to figure out why?
    stars = process_astronomy_data("fake_data.csv")
    assert len(stars) == 2
    assert stars[0].name == "Star A"
    assert stars[1].name == "Star B"


    

.F                                                                                           [100%]
============================================= FAILURES =============================================
___________________________________ test_process_astronomy_data ____________________________________

    def test_process_astronomy_data():
        # This test fails. Can you use the debugger to figure out why?
        stars = process_astronomy_data("fake_data.csv")
>       assert len(stars) == 2
E       assert 0 == 2
E        +  where 0 = len([])

/var/folders/vv/d9ncb4ms2x1gl0mmkrvk8m7c0000gp/T/ipykernel_64744/702681110.py:25: AssertionError
===================================== short test summary info ======================================
FAILED t_6651fbd894764815bb8c2b5f3a5d5116.py::test_process_astronomy_data - assert 0 == 2
